In [1]:
import numpy as np

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
def parse(npz_arr):
    return npz_arr["predictions"], npz_arr["targets"]

def nll(outputs, labels):
    labels = labels.astype(int)
    idx = (np.arange(labels.size), labels)
    ps = outputs[idx]
    nll = -np.sum(np.log(ps))
    return nll

def nll_from_arr(npz_arr, name="", print_name=False):
    all_outputs, labels = parse(npz_arr)
    outputs = np.sum(all_outputs, 2)
    if outputs is None:
        nll = None
    else:
        labels = labels.astype(int)
        idx = (np.arange(labels.size), labels)
        ps = outputs[idx]
        nll = -np.sum(np.log(ps))
    if print_name:
        print(name, end=": ")
    if nll is not None:
        print("%.3f"%nll)
    else:
        print("-")
    return nll

In [3]:
def accuracy(npz_arr, name="", print_name=False):
    outputs, labels = parse(npz_arr)
    
    if outputs is None:
        acc = None
    else:
        pred_probs = np.sum(outputs, 2)
        acc = np.mean(np.argmax(pred_probs, axis=1) == labels)
    if print_name:
        print(name, end=": ")
    if acc is not None:
        print("%.4f"%acc)
    else:
        print("-")
    return acc

In [4]:
npz_locations = 'rank_experiment/swag/'

In [ ]:
swag_nll = []
swag_acc = []
for epoch in range(163,326):
        npz_arr = np.load(npz_locations+str(epoch)+'.npz')
        swag_nll.append( nll_from_arr(npz_arr) / 10000 )
        swag_acc.append( accuracy(npz_arr))

9582.432
0.7223
9021.226
0.7395
8706.013
0.7480
8613.429
0.7484
8480.214
0.7518
8441.128
0.7546
8364.406
0.7551
8316.363
0.7575
8256.242
0.7591
8232.731
0.7575
8225.362
0.7589
8203.508
0.7578
8190.591
0.7600
8183.817
0.7619
8221.173
0.7617
8225.716
0.7630
8166.530
0.7641
8203.158
0.7627
8159.058
0.7612
8186.102
0.7593
8194.011
0.7584
8193.877
0.7604
8189.761
0.7617
8192.972
0.7602
8207.428
0.7601
8174.877
0.7619
8176.803
0.7606
8163.907
0.7622
8144.133
0.7631
8128.235
0.7626
8156.670
0.7660
8088.207
0.7653
8073.617
0.7647
8069.017
0.7659
8077.074
0.7644
8080.191
0.7630
8111.110
0.7634
8132.103
0.7635
8133.351
0.7632
8136.230
0.7636
8146.652
0.7647
8171.862
0.7640
8152.392
0.7651
8169.583
0.7642
8149.300
0.7652
8139.028
0.7648
8107.209
0.7652
8155.878
0.7663
8160.857
0.7659
8140.854
0.7660
8135.840
0.7651
8133.427
0.7641
8128.634
0.7653
8116.496
0.7668
8114.412
0.7667
8110.376
0.7668
8098.219
0.7666
8095.946
0.7666
8089.797
0.7664
8089.205
0.7661
8080.404
0.7672
8086.699
0.7656
8113.646

In [ ]:
swag_nll = np.array(swag_nll)
swag_acc = np.array(swag_acc)

In [ ]:
swa_npz_locations = 'rank_experiment/swa/'
swa_nll = []
swa_acc = []
for epoch in range(163,326):
        npz_arr = np.load(swa_npz_locations+str(epoch)+'.npz')
        swa_nll.append( nll_from_arr(npz_arr) / 10000 )
        swa_acc.append( accuracy(npz_arr) )

In [ ]:
swa_nll = np.array(swa_nll)
swa_acc = np.array(swa_acc)

In [ ]:
ecdf_vals = np.load('rank_experiment/ecdf_ensembles_0121.npz')

In [ ]:
ecdf_preds = ecdf_vals['predictions']
ecdf_targets = ecdf_vals['targets']

In [ ]:
ecdf_vals.keys()

In [ ]:
ecdf_nll = []
ecdf_acc = []
for i in range(ecdf_preds.shape[2]):
    current_predictions = ecdf_preds[:, :, 0:(i+1)].sum(2) / (i+1)
    
    ecdf_nll.append( nll( current_predictions , ecdf_targets) )
    ecdf_acc.append( (np.argmax(current_predictions, 1) == ecdf_targets).mean() )

In [ ]:
ecdf_nll = np.array(ecdf_nll) / ecdf_preds.shape[0]
ecdf_acc = np.array(ecdf_acc)

In [ ]:
plt.plot(range(161, 325), ecdf_nll, label = 'SGD Ensemble')
#plt.plot(range(161, 325), ecdf_vals['sgd_loss'], label='SGD Ind.')
plt.plot(range(163, 326), swag_acc, label='SWA')
plt.plot(range(163, 326), nll_vals, label = 'SWAG - 30 Samples')
plt.axvline(x=161+140, c='black', label='Rank 140')

plt.legend()
plt.grid()
plt.ylim((0.8, 0.92))
plt.xlabel('Epoch')
plt.ylabel('NLL')
plt.savefig('/home/wesley/Documents/Papers/udl_paper_source/icml/plots/pics/prn56_c100_nll_rank.pdf')

In [ ]:
plt.plot(range(161, 325), ecdf_acc, label = 'SGD Ensemble')
#plt.plot(range(161, 325), ecdf_vals['sgd_acc']/100, label='SGD Ind.')
plt.plot(range(163, 326), swag_acc, label = 'SWAG - 30 Samples')
plt.plot(range(163, 326), swa_acc, label='SWA')

plt.axvline(x=161+140, c='black', label='Rank 140')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim((0.75, 0.77))
plt.grid()
plt.legend()
plt.savefig('/home/wesley/Documents/Papers/udl_paper_source/icml/plots/pics/prn56_c100_acc_rank.pdf')

In [ ]:
np.savez(file='/home/wesley/Documents/Papers/udl_paper_source/icml/plots/data/prn56_rank.npz',
        swag={'acc':swag_acc, 'nll':swag_nll},
        swa={'acc':swa_acc, 'nll':swa_nll},
        ecdf={'acc':ecdf_acc, 'nll': ecdf_nll},
        sgd={'acc':ecdf_vals['sgd_acc'], 'nll':ecdf_vals['sgd_loss']})